In [1]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold

from sklearn.metrics import roc_auc_score, precision_recall_curve
from sklearn.metrics import auc as calculate_auc
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


from sklearn.utils import shuffle 
from joblib import load, dump
import numpy as np
import pandas as pd
import os

In [2]:
from chembench import load_data, dataset
from molmap import feature

In [3]:
bitsinfo = feature.fingerprint.Extraction().bitsinfo
fp_types = bitsinfo.Subtypes.unique()
fp_types

array(['MorganFP', 'RDkitFP', 'AtomPairFP', 'TorsionFP', 'AvalonFP',
       'EstateFP', 'MACCSFP', 'PharmacoErGFP', 'PharmacoPFP', 'PubChemFP',
       'MHFP6', 'MAP4'], dtype=object)

In [4]:
from scipy.stats.stats import pearsonr
def r2(y_true, y_pred):
    pcc, _ = pearsonr(y_true,y_pred)
    return pcc[0]**2

def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)  
    return rmse


def PRC_AUC(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

def ROC_AUC(y_true, y_score):
    auc = roc_auc_score(y_true, y_score)
    return auc

In [5]:
hiv = dataset.load_HIV()
bace = dataset.load_BACE()
bbbp = dataset.load_BBBP()
clintox = dataset.load_ClinTox()
sider = dataset.load_SIDER()

datasets = [clintox, sider, bace, bbbp, hiv] #malaria

total samples: 41127
total samples: 1513
total samples: 2039
total samples: 1478
total samples: 1427


In [6]:
performance = []

for data in datasets:
    for fp_type in fp_types:
        task_name = data.task_name
        print(task_name, fp_type)
        df, induces = load_data(task_name)
        X2 = load('/raid/shenwanxiang/10_FP_effect/tempignore/X2_%s_%s.data' % (task_name, fp_type) )
        n, w, c = X2.sum(axis=-1).shape
        X2 = X2.reshape(n, w*c)
        Y = data.y
        if task_name == 'HIV':
            induces = induces[:1]
        
        for sid, idx in enumerate(induces):
            train_idx, valid_idx, test_idx = idx

            X = X2[train_idx]
            y = Y[train_idx]

            X_valid = X2[valid_idx]
            y_valid = Y[valid_idx]

            X_test = X2[test_idx]
            y_test = Y[test_idx] 

            # Set up possible values of parameters to optimize over
            n_neighbors_list = np.arange(1,15,2)
            if task_name == 'HIV':
                n_neighbors_list = np.arange(1, 11, 2)
                
            weights_list =  ['uniform', 'distance']
            res = []
            for n_neighbors in tqdm(n_neighbors_list, ascii=True):
                for weights in weights_list:                
                    clf = KNeighborsClassifier(n_neighbors=n_neighbors, weights = weights)
                    clf.fit(X, y)
                    score = clf.score(X_valid, y_valid)
                    res.append([n_neighbors, weights, score])

            dfr = pd.DataFrame(res, columns = ['n_neighbors', 'weights', 'score'])
            gidx = dfr['score'].idxmax()
            best_params = dfr.iloc[gidx].to_dict()
            best_params.pop('score')
            best_params

            clf = KNeighborsClassifier(**best_params)
            clf.fit(X, y, )
            
            if Y.shape[1] >= 2:
                probs =  clf.predict_proba(X_test)
                test_roc_aucs = []
                for i, y_probs in enumerate(probs):
                    test_roc_auc = ROC_AUC(y_test[:, i],y_probs[:, 1])
                    test_roc_aucs.append(test_roc_auc)
                test_roc_auc = np.nanmean(test_roc_aucs)
                print(test_roc_aucs)
            else:
                test_roc_auc = ROC_AUC(y_test,clf.predict_proba(X_test)[:, 1])
                
            results = {"task_name":task_name, 'fp_type':fp_type,"split-time":sid, "test_roc_auc":test_roc_auc}
            
            print(results)
            performance.append(results)

  0%|          | 0/14 [00:00<?, ?it/s]

ClinTox MorganFP
loading dataset: ClinTox number of split times: 3


  0%|          | 0/14 [00:00<?, ?it/s]

[0.6940222897669706, 0.6004797441364605]
{'task_name': 'ClinTox', 'fp_type': 'MorganFP', 'split-time': 0, 'test_roc_auc': 0.6472510169517156}


  0%|          | 0/14 [00:00<?, ?it/s]

...

{'task_name': 'HIV', 'fp_type': 'PubChemFP', 'split-time': 0, 'test_roc_auc': 0.7147105969601577}
HIV MHFP6
loading dataset: HIV number of split times: 3


100%|##########| 5/5 [1:12:53<00:00, 874.68s/it]


{'task_name': 'HIV', 'fp_type': 'MHFP6', 'split-time': 0, 'test_roc_auc': 0.7592218853203035}
HIV MAP4
loading dataset: HIV number of split times: 3


100%|##########| 5/5 [1:13:03<00:00, 876.44s/it]


{'task_name': 'HIV', 'fp_type': 'MAP4', 'split-time': 0, 'test_roc_auc': 0.6985148419243322}


In [7]:
pd.DataFrame(performance).to_csv('./knn_classification.csv')

In [8]:
pd.DataFrame(performance).groupby(['task_name', 'fp_type'])[['test_roc_auc']].apply(np.mean)

test_roc_auc
task_name fp_type                    
BACE      AtomPairFP         0.833243
          AvalonFP           0.849457
          EstateFP           0.796649
          MACCSFP            0.846196
          MAP4               0.730797
          MHFP6              0.824819
          MorganFP           0.841938
          PharmacoErGFP      0.766033
          PharmacoPFP        0.763859
          PubChemFP          0.812953
          RDkitFP            0.754710
          TorsionFP          0.850362
BBBP      AtomPairFP         0.657722
          AvalonFP           0.614173
          EstateFP           0.672030
          MACCSFP            0.645053
          MAP4               0.618268
          MHFP6              0.632479
          MorganFP           0.622652
          PharmacoErGFP      0.661913
          PharmacoPFP        0.633443
          PubChemFP          0.665430
          RDkitFP            0.654350
          TorsionFP          0.630937
ClinTox   AtomPairFP         0.740678
          AvalonFP           0.664326
          EstateFP           0.745102
          MACCSFP            0.732980
          MAP4               0.607512
          MHFP6              0.713858
          MorganFP           0.652936
          PharmacoErGFP      0.774105
          PharmacoPFP        0.783614
          PubChemFP          0.703787
          RDkitFP            0.681279
          TorsionFP          0.639916
HIV       AtomPairFP         0.695667
          AvalonFP           0.698641
          EstateFP           0.687594
          MACCSFP            0.680781
          MAP4               0.698515
          MHFP6              0.759222
          MorganFP           0.705185
          PharmacoErGFP      0.756114
          PharmacoPFP        0.707640
          PubChemFP          0.714711
          RDkitFP            0.746503
          TorsionFP          0.704668
SIDER     AtomPairFP         0.624285
          AvalonFP           0.624734
          EstateFP           0.612964
          MACCSFP            0.603737
          MAP4               0.586263
          MHFP6              0.626298
          MorganFP           0.621006
          PharmacoErGFP      0.594410
          PharmacoPFP        0.628017
          PubChemFP          0.617368
          RDkitFP            0.624938
          TorsionFP          0.629307